In [35]:
def get_overall_reviews(driver, key_metrics, df):
    while True:
        
        driver.execute_script("window.scrollTo(1300, 1850);")
        try:
            elements = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located(
                    (By.CSS_SELECTOR, "div[class = 'v2__EIReviewDetailsV2__fullWidth v2__EIReviewDetailsV2__clickable']")))
        except:
            pass
        else:
            for i in range(0, len(elements)):
                    try:
                        elements[i].click()
                    except NoSuchElementException:
                        print("Element is not clickable")
                    else:
                        elements[i].click()
        finally:
            page = driver.page_source
            soup = BeautifulSoup(page)
            review_lst = soup.find_all('li', {'class':'noBorder empReview cf pb-0 mb-0'})
            for i in range(0, len(review_lst)):

                reviews = review_lst[i]
                title = reviews.find('h2', {"class":'mb-xxsm mt-0 css-93svrw el6ke055'}).get_text()
                status = reviews.find('span', {"class":'pt-xsm pt-md-0 css-1qxtz39 eg4psks0'}).get_text()
                date_position_txt = reviews.find('span', {'class':'middle common__EiReviewDetailsStyle__newGrey'}).get_text()
                pros = reviews.find('span', {'data-test':"pros"}).get_text()
                cons = reviews.find('span', {'data-test':"cons"}).get_text()
                try:
                    advice_to_mgmt = reviews.find('span', {'data-test':"advice-management"}).get_text()
                except:
                    advice_to_mgmt = None
                helpfulness = reviews.find('div', {'class':"common__EiReviewDetailsStyle__socialHelpfulcontainer pt-std"}).get_text()
                rating = reviews.find('span', {'class':'ratingNumber mr-xsm'}).get_text()

                df.loc[len(df)] = [title, status, date_position_txt, pros, cons, advice_to_mgmt, helpfulness, rating]

            # driver.maximize_window()
            # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # go to the next page
            try:
                next_button = WebDriverWait(driver, 20).until(EC.presence_of_element_located(
                    (By.XPATH, '//*[@id="Container"]/div[1]/div[2]/main/div[6]/div/div[1]/button[2]/span')))

            except NoSuchElementException:
                print('no next button')
                break

            else:
                next_button.click()
                time.sleep(5)
                # WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located(
                #     (By.CSS_SELECTOR, "div[class = 'v2__EIReviewDetailsV2__fullWidth v2__EIReviewDetailsV2__clickable']")))
                                       
    return df

In [4]:
def position_type(x):

    position_type_lst = ['engineer', 'marketing', 'finance', 'sale', 'administrative', 'operation'
                         'consultant', 'analyst', 'human resource', 'product', 'director',
                         'account executive', 'data', 'user experience', 'recruiter' ,'dev', 'project']
    try: 
        x = x.lower()
    except:
        return None
    else:
        for ele in position_type_lst:
            if ele in x:
                return ele
            elif ele != position_type_lst:
                continue
            else:
                return None        


In [9]:
def get_benefit_reviews(key_metrics, df):
    while True:
        page = driver.page_source
        soup = BeautifulSoup(page)
        review_lst = soup.find_all('div', {'class':'pt-sm css-1ecrgor'})
        for i in range(0, len(review_lst)):

            reviews = review_lst[i]
            date = reviews.find('span', {'class':'css-1uyte9r', 'data-test':'review-date'}).get_text()
            text = reviews.find('p', {'class':'mt-std mb-std mt-sm-xsm'}).get_text()
            rating = reviews.find('strong', {'class':'mr-xxsm css-b63kyi'}).get_text()
            position = reviews.find('div', {'class':'mt-xxsm mt-sm-0 ml-0 ml-sm-xsm css-1uyte9r'}).get_text()
            df.loc[len(df)] = [date, text, rating, position]

        driver.maximize_window()
        # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # go to the next page
        try:
            next_button = driver.find_element(By.XPATH, '//*[@id="NodeReplace"]/div/div[3]/div[2]/div/div[1]/button[2]')
        
        except NoSuchElementException:
            break
        
        else:
            next_button.click()

    return df

In [1]:
def preprocess_text(text: str, remove_stopwords: bool) -> str:
    """This utility function sanitizes a string by:
    - removing links
    - removing special characters
    - removing numbers
    - removing stopwords
    - transforming in lowercase
    - removing excessive whitespaces
    Args:
        text (str): the input text you want to clean
        remove_stopwords (bool): whether or not to remove stopwords
    Returns:
        str: the cleaned text
    """

    # remove links
    text = re.sub(r"http\S+", "", text)
    # remove special chars and numbers
    text = re.sub("[^A-Za-z]+", " ", text)
    # remove stopwords
    if remove_stopwords:
        # 1. tokenize
        tokens = nltk.word_tokenize(text)
        # 2. check if stopword
        tokens = [w for w in tokens if not w.lower() in stopwords.words("english")]
        # 3. join back together
        text = " ".join(tokens)
    # return text in lower case and stripped of whitespaces
    text = text.lower().strip()
    return text

In [ ]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

In [ ]:
def status(x):
    if 'Current' in x:
        return 'Current'
    elif 'Former' in x:
        return 'Former'
    else:
        return None

In [ ]:
def position(x):
    if 'Current Employee' in x or 'Former Employee' in x:
        return 'unspecified'
    else:
        return x

In [1]:
def clean_years(x):
    if x is not np.nan:
        if 'less than 1 year' not in x:
            return int(x.split(' ')[3])
        else:
            return float(0.5)
    else:
        return x

In [ ]:
def convert_type(x):
    if type(x) == str and x != '[]':
        return float(x.strip('[]'))
    elif x == '[]':
        return None
    else:
        return None

In [ ]:
def get_star_rating_dict(driver, star_rating_dict):
    
    page = driver.page_source
    soup = BeautifulSoup(page)
    css_color = soup.find_all('style')
    color_lst = []
    for i in range(0, len(css_color)):
        if 'linear-gradient' in css_color[i].get_text():
            color_lst.append(css_color[i])
            
    color_lst = list(set(color_lst))
    
    for i in range(0, len(color_lst)):

        if '#0caa41 ' in color_lst[i].get_text():
            star_rating_id = color_lst[i].get_text().partition('{')[0].strip('.')
            star_rating_value = float(color_lst[i].get_text().partition('#0caa41 ')[2].split('%,')[0])/100
            if star_rating_id not in star_rating_dict.keys():
                star_rating_dict[star_rating_id] = star_rating_value
        else:
            continue
            
    return star_rating_dict

In [ ]:
def additional_star_ratings(driver):
    count = 0
    all_star_rating_info = []
    title_lst = []
    star_rating_dict = {}
    
    while count <= 258:
        driver.execute_script("window.scrollTo(1300, 1850);")        
        page = driver.page_source
        soup = BeautifulSoup(page)
        star_rating_dict = get_star_rating_dict(driver, star_rating_dict)

        review_lst = soup.find_all('li', {'class':'noBorder empReview cf pb-0 mb-0'})
        for i in range(0, len(review_lst)):
            star_rating = {}

            reviews = review_lst[i]
            title = reviews.find('h2', {"class":'mb-xxsm mt-0 css-93svrw el6ke055'}).get_text()
            title_lst.append(title)
            tooltip = reviews.find('div', {'class': 'tooltipContainer'})
            if tooltip != None:
                metrics_6 = tooltip.find_all('li')
                for i in range(0, len(metrics_6)):
                    metric_rating_key = metrics_6[i].find_all('div')[0].get_text()
                    metric_rating_value = metrics_6[i].find_all('div')[1]['class'][0]
                    star_rating[metric_rating_key] = [v for k, v in star_rating_dict.items() if metric_rating_value in k]

            else:
                metrics_name_lst = ['Work/Life Balance','Culture & Values',
                                     'Diversity & Inclusion','Career Opportunities',
                                     'Compensation and Benefits','Senior Management']
                for name in metrics_name_lst:
                    star_rating[name] = None

            all_star_rating_info.append(star_rating)
            
        try:
            next_button = WebDriverWait(driver, 20).until(EC.presence_of_element_located(
                    (By.XPATH, '//*[@id="Container"]/div[1]/div[2]/main/div[6]/div/div[1]/button[2]/span')))
            
        except NoSuchElementException:
            print('no next button')
            break

        else:
            if next_button.is_enabled():
                next_button.click()
                count += 1
                time.sleep(3)
            else:
                break

    return pd.DataFrame(all_star_rating_info), title_lst